In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Get the data and read the data(get rid of the columns that are empty/NaN as values using pd.NaN or dropping them)

In [4]:
reg_season = pd.read_csv('files/reg_season_2022_data.csv')
playoffs = pd.read_csv('files/playoffs_2022_data.csv')
reg_season = reg_season.drop(['RANK'], axis = 1)
playoffs = playoffs.drop(['RANK'], axis = 1)

Find out the Columns and which information is necessary in this analysis

In [5]:
reg_season.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 716 entries, 0 to 715
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FULL NAME        716 non-null    object 
 1   TEAM             716 non-null    object 
 2   POS              716 non-null    object 
 3   AGE              716 non-null    float64
 4   GP               716 non-null    int64  
 5   MPG              716 non-null    float64
 6   MIN%r            716 non-null    float64
 7   USG%             716 non-null    float64
 8   TO%              703 non-null    float64
 9   FTA              716 non-null    int64  
 10  FT%              716 non-null    float64
 11  2PA              716 non-null    int64  
 12  2P%              716 non-null    float64
 13  3PA              716 non-null    int64  
 14  3P%              716 non-null    float64
 15  eFG%             701 non-null    float64
 16  TS%              703 non-null    float64
 17  PPG             

In [6]:
reg_season

,FULL NAME,TEAM,POS,AGE,GP,MPG,MIN%r,USG%,TO%,FTA,...,RPG,TRB%,APG,AST%,SPG,BPG,TOPG,VI(Versatility),ORTG,DRTG.
0,Precious Achiuwa,Tor,F,22.56,73,23.6,49.2,18.5,11.3,131,...,6.5,14.9,1.1,6.9,0.51,0.56,1.15,6.8,105.4,104.0
1,Steven Adams,Mem,C,28.73,76,26.3,54.8,12.0,19.6,199,...,10.0,19.9,3.4,16.1,0.87,0.79,1.51,9.4,124.7,103.9
2,Bam Adebayo,Mia,C-F,24.73,56,32.6,67.9,25.0,14.4,340,...,10.1,17.5,3.4,17.5,1.43,0.79,2.64,10.7,117.2,98.2
3,Santi Aldama,Mem,F-C,21.25,32,11.2,23.4,18.5,9.9,32,...,2.7,12.6,0.7,7.7,0.19,0.31,0.50,7.1,101.1,103.9
4,LaMarcus Aldridge,Bro,C-F,36.73,47,22.3,46.5,22.4,8.0,102,...,5.5,13.4,0.9,6.3,0.30,1.00,0.94,7.2,119.6,108.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,Thaddeus Young,Tor,F,33.80,26,18.3,38.1,16.0,12.4,27,...,4.4,13.0,1.7,13.6,1.19,0.42,0.85,7.9,112.8,102.0
712,Trae Young,Atl,G,23.56,76,34.9,72.7,34.4,14.5,553,...,3.7,5.9,9.7,46.8,0.96,0.09,3.99,11.5,119.5,114.3
713,Omer Yurtseven,Mia,C,23.81,56,12.6,26.3,19.8,13.0,61,...,5.3,23.6,0.9,11.0,0.30,0.36,0.73,9.3,113.3,94.7
714,Cody Zeller,Por,F-C,29.51,27,13.1,27.4,16.0,14.6,49,...,4.6,19.3,0.8,9.4,0.30,0.22,0.70,8.2,128.3,101.8


To qualify for All-NBA status for a given NBA season, players should play at least 70% of their teams games(58/82 games), so we're going to predict the chance that a player can reach All-NBA status(15 per year). So we filter out players who have played 58 or more games and use different metrics to see how likely it is that we can calculate the best players in the league

In [7]:
reg_season = reg_season[reg_season.get('GP') >= 58]
reg_season

,FULL NAME,TEAM,POS,AGE,GP,MPG,MIN%r,USG%,TO%,FTA,...,RPG,TRB%,APG,AST%,SPG,BPG,TOPG,VI(Versatility),ORTG,DRTG.
0,Precious Achiuwa,Tor,F,22.56,73,23.6,49.2,18.5,11.3,131,...,6.5,14.9,1.1,6.9,0.51,0.56,1.15,6.8,105.4,104.0
1,Steven Adams,Mem,C,28.73,76,26.3,54.8,12.0,19.6,199,...,10.0,19.9,3.4,16.1,0.87,0.79,1.51,9.4,124.7,103.9
7,Grayson Allen,Mil,G,26.51,66,27.3,57.0,15.2,6.7,74,...,3.4,6.5,1.5,7.6,0.70,0.27,0.65,5.6,121.7,110.7
12,Kyle Anderson,Mem,F-G,28.56,69,21.5,44.8,16.2,12.1,105,...,5.3,12.9,2.7,16.2,1.12,0.67,1.03,8.9,109.3,103.3
13,Giannis Antetokounmpo,Mil,F,27.35,67,32.9,68.5,34.9,12.2,766,...,11.6,18.7,5.8,31.7,1.07,1.36,3.27,15.3,123.9,100.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,Ziaire Williams,Mem,F,20.58,62,21.7,45.2,14.9,8.4,55,...,2.1,5.0,1.0,6.3,0.58,0.18,0.66,4.7,112.9,107.1
704,Christian Wood,Hou,F,26.54,68,30.8,64.1,23.3,11.0,332,...,10.1,18.0,2.3,12.2,0.78,0.97,1.85,9.7,114.1,108.1
706,Delon Wright,Atl,G,29.96,77,18.9,39.3,10.3,13.2,63,...,2.9,8.3,2.4,16.6,1.21,0.25,0.58,6.6,126.2,112.3
712,Trae Young,Atl,G,23.56,76,34.9,72.7,34.4,14.5,553,...,3.7,5.9,9.7,46.8,0.96,0.09,3.99,11.5,119.5,114.3


As seen, there are only around 205 players who actually played 58 or more games in the season, so in this case, one of the best abilities is availability

One of the common metrics that people use in calculating how well players do in certain games is called 'game score'. To calculate game score, use this formula    

In [8]:
reg_season.columns

Index(['FULL NAME', 'TEAM', 'POS', 'AGE', 'GP', 'MPG', 'MIN%r', 'USG%', 'TO%',
       'FTA', 'FT%', '2PA', '2P%', '3PA', '3P%', 'eFG%', 'TS%', 'PPG', 'RPG',
       'TRB%', 'APG', 'AST%', 'SPG', 'BPG', 'TOPG', 'VI(Versatility)', 'ORTG',
       'DRTG.'],
      dtype='object')

In [9]:
def game_score_formula(pts, fg, fga, fta, ft, orb, drb, stl, ast, blk, pf, tov):
    """ 
    A measure of how well a player is doing in a single game.
    Created by John Hollinger to give rough measure of how well each
    singular player did in a game(40 is a great game, 10 is average)
    """
    return (pts) + (0.4 * fg) - (0.7 * fga) + (0.4 * (fta-ft)) + (0.7 * orb) + (0.3 * drb) + stl + (0.7 * ast) + (0.7 * blk) - (0.4 * pf) - tov

In [10]:
reg_season['game_score'] = reg_season.get(['GP', 'PPG', '2PA', '2P%', '3PA', '3P%', 'FTA', 'FT%', 'ORB'])
playoffs['game_score']

/var/folders/lz/89b0m93557x4pjgtj7v1dcpc0000gn/T/ipykernel_64627/3659460444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reg_season['game_score'] = reg_season.get(['GP', 'PPG', '2PA', '2P%', '3PA', '3P%', 'FTA', 'FT%', 'ORB'])


KeyError: 'game_score'